In [1]:
from mxnet import autograd, nd

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
# 加入噪声
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [3]:
from mxnet.gluon import data as gdata

batch_size = 10
dataset = gdata.ArrayDataset(features, labels)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [4]:
for X, y in data_iter:
    print(X, y)
    break

(
[[ 0.20362946  0.6474341 ]
 [-0.65785325 -1.5830694 ]
 [ 2.0246542   0.9592536 ]
 [-0.37976772  0.71671826]
 [ 0.86399084 -0.9599424 ]
 [-0.30006385  0.5095682 ]
 [-2.165552   -0.91337925]
 [ 0.5234586   0.13859516]
 [-1.5869852  -0.47912037]
 [-0.08212578 -0.05052492]]
<NDArray 10x2 @cpu(0)>, 
[2.402583  8.265481  4.9905815 1.002221  9.20564   1.8576508 2.9953463
 4.7739687 2.6462026 4.2175794]
<NDArray 10 @cpu(0)>)


# 定义模型

In [5]:
from mxnet.gluon import nn

net = nn.Sequential()

In [6]:
net.add(nn.Dense(1))

In [7]:
net

Sequential(
  (0): Dense(None -> 1, linear)
)

# 定义模型

In [8]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))
net.initialize()

D:\Anaconda\lib\site-packages\mxnet\gluon\parameter.py:686: UserWarning: Parameter 'dense0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


# 定义损失函数

In [9]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()

# 定义优化算法

In [10]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# 训练模型

In [11]:
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    print("epoch %d, loss: %f"
          % (epoch, loss(net(features), labels).mean().asnumpy()))

epoch 1, loss: 0.035858
epoch 2, loss: 0.000126
epoch 3, loss: 0.000049
epoch 4, loss: 0.000048
epoch 5, loss: 0.000048


In [12]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.999533 -3.39973 ]]
 <NDArray 1x2 @cpu(0)>)

In [13]:
true_b, dense.bias.data()

(4.2, 
 [4.200085]
 <NDArray 1 @cpu(0)>)